In [32]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
nltk.download('punkt')
nltk.download('wordnet')

import numpy as np
from keras.models import Sequential
from keras.layers import Dense ,Activation , Flatten, Dropout
from keras.optimizers import SGD
import random

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package punkt to /home/xikram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/xikram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [34]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w , intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [35]:
#lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(len(documents) , "documents"), 
print(len(classes) , "classes" , classes)
print(len(words) , "unique lemmatized words" , words)

pickle.dump(words , open("words.pkl",  'wb'))
pickle.dump(classes , open('classes.pkl' ,'wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [36]:
training = []
output_empty = [0]*len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        
        training.append([bag, output_row])
    
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [42]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model =  Sequential()
model.add(Dense(128 , input_shape = (len(train_x[0]),) , activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64 , activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]) , activation ='softmax'))

sgd = SGD(lr =0.01 , decay= 1e-6 , momentum= 0.9 , nesterov= True)
model.compile(loss = 'categorical_crossentropy' , optimizer = sgd , metrics = ['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y) , epochs = 200 , batch_size = 5 , verbose = 1 )
model.save('chatbot.h5')
print("Model Created")    



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/200
4136/4136 [==============================] - 3s 714us/step - loss: 0.4223 - acc: 0.8743
Epoch 2/200
4136/4136 [==============================] - 1s 253us/step - loss: 0.0335 - acc: 0.9932
Epoch 3/200
4136/4136 [==============================] - 1s 228us/step - loss: 0.0216 - acc: 0.9949
Epoch 4/200
4136/4136 [==============================] - 1s 222us/step - loss: 0.0142 - acc: 0.9981
Epoch 5/200
4136/4136 [==============================] - 1s 225us/step - loss: 0.0092 - acc: 0.9985
Epoch 6/200
4136/4136 [==============================] - 1s 233us/step - loss: 0.0100 - acc: 0.9964
Epoch 7/200
4136/4136 [==============================] - 1s 217us/step - loss: 0.0085 - acc: 0.9983
Epoch 8/200
4136/4136 [==============================] - 1s 230us/step - loss: 0.0079 - acc: 0.9976
Epoch 9/200
4136/4136 [==============================] - 1s 232us/step - loss: 0.0050 - acc: 0.9990
Epoc

4136/4136 [==============================] - 1s 243us/step - loss: 0.0017 - acc: 0.9998
Epoch 75/200
4136/4136 [==============================] - 1s 251us/step - loss: 0.0015 - acc: 0.9993
Epoch 76/200
4136/4136 [==============================] - 1s 248us/step - loss: 0.0013 - acc: 0.9998
Epoch 77/200
4136/4136 [==============================] - 1s 280us/step - loss: 5.6659e-04 - acc: 1.0000
Epoch 78/200
4136/4136 [==============================] - 1s 238us/step - loss: 0.0011 - acc: 0.9998
Epoch 79/200
4136/4136 [==============================] - 1s 278us/step - loss: 6.0181e-04 - acc: 1.0000
Epoch 80/200
4136/4136 [==============================] - 1s 237us/step - loss: 0.0013 - acc: 0.9995
Epoch 81/200
4136/4136 [==============================] - 1s 256us/step - loss: 0.0018 - acc: 0.9995
Epoch 82/200
4136/4136 [==============================] - 1s 250us/step - loss: 5.0112e-04 - acc: 1.0000
Epoch 83/200
4136/4136 [==============================] - 1s 213us/step - loss: 7.1698e-04 -

4136/4136 [==============================] - 1s 220us/step - loss: 4.7942e-04 - acc: 1.0000
Epoch 153/200
4136/4136 [==============================] - 1s 218us/step - loss: 2.6546e-04 - acc: 1.0000
Epoch 154/200
4136/4136 [==============================] - 1s 213us/step - loss: 8.4925e-04 - acc: 0.9998
Epoch 155/200
4136/4136 [==============================] - 1s 218us/step - loss: 0.0014 - acc: 0.9998
Epoch 156/200
4136/4136 [==============================] - 1s 216us/step - loss: 4.3410e-04 - acc: 1.0000
Epoch 157/200
4136/4136 [==============================] - 1s 210us/step - loss: 2.3638e-04 - acc: 1.0000
Epoch 158/200
4136/4136 [==============================] - 1s 218us/step - loss: 4.0116e-04 - acc: 1.0000
Epoch 159/200
4136/4136 [==============================] - 1s 218us/step - loss: 3.5448e-04 - acc: 1.0000
Epoch 160/200
4136/4136 [==============================] - 1s 207us/step - loss: 5.0057e-04 - acc: 1.0000
Epoch 161/200
4136/4136 [==============================] - 1s 22